# Main advantages of Hierarchical CLustering:
- Do not have to assume any particular number of clusters, any desired number of clusters can be obtained cuttin the endogram
- They may correspond to meaningful taxonomies: gerarchical classification of objects

## Two main types
1) Agglomerative: start with each poin as indivudual clustes and merge clustest while we have k clusters
2) Divisive: start with all-inclusive cluster and split while we have k clustes

In both cases we have to choose a distance measure and split/merge one cluster at a time, selecting the two clusters that have the minumum distance

## Inter-Cluster Distane
- MIN (single link): closest points in the two clusters
    - take strongly into account the contiguity so it can handle globular shapes but it is sentitive to noise
- MAX (complete link): farther points in the two clusters
    - less subsceptible to noise but tends to break large clusters and is biased towards globular ones
- Group average: mean of all possible distances between the point of the two clusters
    - good compromise between single and complete link, still a bit biased toward globular clusters
- Distance between centroids: distance between the centroids of the two clusters
- Ward's Method

In [1]:
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram
import pandas as pd

In [2]:
df = pd.read_csv("./dataset/merged_df_dataset_cleaned_with_indicators.csv")

/tmp/ipykernel_34589/408071329.py:1: DtypeWarning: Columns (0,1,2,4,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv("./dataset/merged_df_dataset_cleaned_with_indicators.csv")


As a first step, since we will work on numerical features we remove categorical/nominal? attributes 

In [8]:
df.drop(["user_id", "name", "lang", "bot", "account_created", "tweet_id", "tweet_created", "text"], axis=1)

,statuses_count,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,account_age_in_days,account_average_tweets_per_day,tweet_age_in_days,text_length,special_char_in_text
0,626.0,0.0,0.0,0.0,0.0,0.0,0.0,3853.0,0.162471,3854.0,32.0,6.0
1,626.0,0.0,0.0,0.0,0.0,0.0,0.0,3853.0,0.162471,3821.0,20.0,5.0
2,626.0,0.0,0.0,0.0,0.0,0.0,0.0,3853.0,0.162471,3821.0,17.0,0.0
3,626.0,0.0,0.0,0.0,0.0,0.0,0.0,3853.0,0.162471,3820.0,103.0,29.0
4,626.0,0.0,0.0,0.0,0.0,0.0,0.0,3853.0,0.162471,3819.0,25.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
18887411,68.0,5.0,0.0,0.0,0.0,0.0,1.0,2068.0,0.032882,879.0,62.0,13.0
18887412,68.0,0.0,0.0,0.0,0.0,1.0,0.0,2068.0,0.032882,879.0,28.0,7.0
18887413,68.0,0.0,0.0,0.0,1.0,0.0,0.0,2068.0,0.032882,879.0,24.0,5.0
18887414,68.0,862.0,0.0,0.0,1.0,1.0,1.0,2068.0,0.032882,879.0,87.0,22.0
